In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm
from tqdm.asyncio import tqdm

from prompts import load_template
from utils import DataVersionManager

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"
CATEGORY = "category"
ANSWER = "answer"
REASONING = "reasoning"

dvm = DataVersionManager()

In [ ]:
# 현재 실험은 1.3.x 입니다.
df = dvm.get_integrated_experiment_data()
df = df[df[CATEGORY] == "외적 추론"]
df.head()

,id,paragraph,question,choices,answer,question_plus,reasoning,category,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_1_page,keyword_2_page,keyword_3_page,keyword_4_page,keyword_5_page
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...,외적 추론,송준길,서인,예송논쟁,인조반정,탕평 정치,"송준길(宋浚吉, 1606년 12월 28일 ~ 1672년 12월 2일)은 조선시대 후...","서인(西人)은 조선 시대 붕당정치 초기의 당파로, 1575년(선조 8)의 동서분당(...","예송(禮訟, 문화어: 례송) 또는 예송논쟁(禮訟論爭, 문화어: 례송론쟁)은 예절에 ...","인조반정(仁祖反正)은 1623년 4월 11일(음력 3월 12일) 김류, 이귀, 신경...",NaN
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)...",외적 추론,삼한,고려사,동녕부,몽골,지눌,"삼한(三韓)은 삼국 시대 이전, 곧 원삼국 시대에 한반도에 자리 잡고 있던 마한(馬...",고려사(高麗史)는 조선 전기에 편찬된 고려 왕조의 역사를 저술한 역사서이며 기전체(...,동녕부(東寧府)는 원나라가 고려 서경을 편입한 후 설치한 통치기관이다.\n\n역사\...,"몽골국(몽골어: Монгол Улс, ᠮᠣᠩᠭᠣᠯᠤᠯᠤᠰ 멍걸 올스 듣기 , 음차:...","지눌(知訥, 1158년~1210년 4월 22일(음력 3월 27일))은 고려 중기 ~..."
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...,외적 추론,소정방,김문영,신라,김춘추,백제,"소정방(蘇定方, 592년 ~ 667년)은 당나라의 장군이다. 소열(蘇烈)이 본명이지...","김문영(金文穎, 생몰년 미상)은 7세기 신라의 장군이자 관인이었다. 김문영(金文永)...","신라(新羅)는 기원전 57년부터 기원후 935년까지 고구려, 백제와 함께 고대 한반...","태종 무열왕(太宗武烈王, 603년 ~ 661년 음력 6월)은 신라(新羅)의 제29대...","백제(百濟)는 한국의 고대 국가 중 하나로, 고구려, 신라와 함께 삼국 시대를 구성..."
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...,외적 추론,영조,서원,대전회통,비변사,통상 수교,"영조(英祖, 1694년 10월 31일(음력 9월 13일) ~ 1776년 4월 22일...",서원(書院)는 중국에서 발단 한자 문화권의 동아시아 (일본 한국 베트남)에 파급된...,《대전회통》(大典會通)은 1865년(고종 2년)에 조두순(趙斗淳) 등이 왕명에 따라...,비변사(備邊司)는 조선시대 군국기무(軍國機務)를 관장한 문무합의기구(文武合議機構)다...,NaN
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...,외적 추론,대무예,고구려,발해,3성 6부,중앙집권,"무왕(武王, ?~737년)은 발해의 제2대 왕이다. 휘는 무예(武藝)이다. 연호는 ...",고구려(高句麗)는 한국의 고대 왕조국가 중 하나이다. 국성은 고씨(高氏)이다. 본래...,발해(渤海) 또는 발해국(渤海國)은 고구려 멸망 이후 고구려 계승을 표방하던 대조영...,삼성육부제(三省六部制)는 과거 중세 중국 왕조의 중앙 정치 제도이다. 이는 주변국에...,"중앙집권(中央集權, 영어: centralisation, centralization)..."


In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="document_summarization.txt", template_type="paragraph_generation")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

summarizer = prompt | llm

In [ ]:
def summarize(row: pd.Series, i: int) -> str:
    """
    위키피디아 문서를 요약하는 함수

    Args:
        row (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나
        i {int}: 참고할 키워드

    Returns:
        str: 추출된 5개 키워드
    """

    keywords_list = [row[f"keyword_{idx}"] for idx in range(1, 6)]
    keywords = ", ".join(keywords_list)

    if pd.isna(row[f"keyword_{i}_page"]):
        return ""
    document = row[f"keyword_{i}_page"]

    response = summarizer.invoke(
        {
            "keywords": keywords,
            "document": document,
        },
    )
    return response.content


tqdm.pandas()
for i in range(1, 6):
    df[f"keyword_{i}_summary"] = df.progress_apply(lambda row: summarize(row, i), axis=1)

df.head()

100%|██████████| 639/639 [54:24<00:00,  5.11s/it]  


,id,paragraph,question,choices,answer,question_plus,reasoning,category,keyword_1,keyword_2,...,keyword_1_page,keyword_2_page,keyword_3_page,keyword_4_page,keyword_5_page,keyword_1_summary,keyword_2_summary,keyword_3_summary,keyword_4_summary,keyword_5_summary
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...,외적 추론,송준길,서인,...,"송준길(宋浚吉, 1606년 12월 28일 ~ 1672년 12월 2일)은 조선시대 후...","서인(西人)은 조선 시대 붕당정치 초기의 당파로, 1575년(선조 8)의 동서분당(...","예송(禮訟, 문화어: 례송) 또는 예송논쟁(禮訟論爭, 문화어: 례송론쟁)은 예절에 ...","인조반정(仁祖反正)은 1623년 4월 11일(음력 3월 12일) 김류, 이귀, 신경...",NaN,"송준길(宋浚吉, 1606년 12월 28일 ~ 1672년 12월 2일)은 조선 후기의...","송준길(宋浚吉, 1606~1672)은 조선 후기 서인(西人) 정치가이자 유학자로, ...","송준길(宋浚吉, 1606~1672)은 조선 후기의 서인(西人) 학자이자 정치가로, ...","송준길(宋浚吉, 1606~1672)은 조선 후기의 서인(西人) 정치가이자 유학자로,...",
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)...",외적 추론,삼한,고려사,...,"삼한(三韓)은 삼국 시대 이전, 곧 원삼국 시대에 한반도에 자리 잡고 있던 마한(馬...",고려사(高麗史)는 조선 전기에 편찬된 고려 왕조의 역사를 저술한 역사서이며 기전체(...,동녕부(東寧府)는 원나라가 고려 서경을 편입한 후 설치한 통치기관이다.\n\n역사\...,"몽골국(몽골어: Монгол Улс, ᠮᠣᠩᠭᠣᠯᠤᠯᠤᠰ 멍걸 올스 듣기 , 음차:...","지눌(知訥, 1158년~1210년 4월 22일(음력 3월 27일))은 고려 중기 ~...","삼한(三韓)은 원삼국 시대에 한반도 중남부에 자리 잡고 있던 마한(馬韓), 진한(辰...","고려사(高麗史)는 조선 전기에 편찬된 고려 왕조의 역사서로, 기전체(紀傳體) 형식으...","동녕부(東寧府)는 원나라가 고려의 서경을 편입한 후 설치한 통치기관으로, 1269년...","몽골국(몽골 올스)은 동아시아의 내륙국으로, 면적은 1,564,116km²이며 인구...","지눌(知訥, 1158~1210)은 고려 중기에서 후기의 승려로, 불교의 선과 교를 ..."
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...,외적 추론,소정방,김문영,...,"소정방(蘇定方, 592년 ~ 667년)은 당나라의 장군이다. 소열(蘇烈)이 본명이지...","김문영(金文穎, 생몰년 미상)은 7세기 신라의 장군이자 관인이었다. 김문영(金文永)...","신라(新羅)는 기원전 57년부터 기원후 935년까지 고구려, 백제와 함께 고대 한반...","태종 무열왕(太宗武烈王, 603년 ~ 661년 음력 6월)은 신라(新羅)의 제29대...","백제(百濟)는 한국의 고대 국가 중 하나로, 고구려, 신라와 함께 삼국 시대를 구성...","소정방(蘇定方, 592년 ~ 667년)은 당나라의 저명한 장군으로, 본명은 소열(蘇...","김문영(金文穎, 생몰년 미상)은 7세기 신라의 장군이자 관인으로, 태종 무열왕 7년...","신라(新羅)는 기원전 57년부터 기원후 935년까지 존재한 고대 한반도의 국가로, ...","태종 무열왕(太宗武烈王, 603년 ~ 661년)은 신라의 제29대 왕으로, 재위 기...","백제(百濟)는 한국의 고대 국가 중 하나로, 고구려와 신라와 함께 삼국 시대를 구성..."
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...,외적 추론,영조,서원,...,"영조(英祖, 1694년 10월 31일(음력 9월 13일) ~ 1776년 4월 22일...",서원(書院)는 중국에서 발단 한자 문화권의 동아시아 (일본 한국 베트남)에 파급된...,《대전회통》(大典會通)은 1865년(고종 2년)에 조두순(趙斗淳) 등이 왕명에 따라...,비변사(備邊司)는 조선시대 군국기무(軍國機務)를 관장한 문무합의기구(文武合議機構)다...,NaN,"영조(英祖, 1694년 10월 31일 ~ 1776년 4월 22일)는 조선의 제21대...","영조(英祖) 시대의 서원(書院)은 조선 중기 이후 설립된 사설 교육 기관으로, 유교...","영조(英祖)는 조선의 21대 왕으로, 그의 통치 기간 동안 여러 가지 중요한 정책과...","영조(英祖)는 조선시대의 왕으로, 그의 통치 기간 동안 여러 중요한 정치적 변화와 ...",
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...,외적 추론,대무예,고구려,...,"무왕(武王, ?~737년)은 발해의 제2대 왕이다. 휘는 무예(武藝)이다. 연호는 ...",고구려(高句麗)는 한국의 고대 왕조국가 중 하나이다. 국성은 고씨(高氏)이다. 본래...,발해(渤海) 또는 발해국(渤海國)은 고구려 멸망 이후 고구려 계승을 표방하던 대조영...,삼성육부제(三省六部制)는 과거 중세 중국 왕조의 중앙 정치 제도이다. 이는 주변국에...,"중앙집권(中央集權, 영어: centralisation, centralization)...","대무예(大武藝, ?~737년)는 발해의 제2대 왕으로, 고왕 대조영의 장남이다. 그...","고구려(高句麗)는 한국의 고대 왕조국가 중 하나로, 기원전 37년 부여의 주몽이 졸...",발해(渤海) 또는 발해국(渤海國)은 고구려 멸망 이후 대조영이 698년에 건국하여 ...,"삼성육부제(三省六部制)는 중세 중국 왕조의 중앙 정치 제도로, 고구려와 발해, 고려...","고구려와 발해는 중앙집권적 정치 체제를 갖춘 고대 국가로, 이들은 대무예(大武藝)라..."


In [ ]:
# 현재 실험은 1.3.x 입니다.
df.to_csv(dvm.save_experiment_result(1, 3), index=False)